<a href="https://colab.research.google.com/github/jagannath311/DataScience-/blob/master/sentanaly_mincut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##the required packages
import os
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import collections
##loading files path from drive
files=os.listdir(r'/content/drive/My Drive/txt_sentoken/pos')
files1=os.listdir(r'/content/drive/My Drive/txt_sentoken/neg')

# Building the subjectivitydescriptor with files from http://www.cs.cornell.edu/people/pabo/movie-review-data/ sentence corpus version 1.0

In [0]:
t=open('rt-polarity.pos','r',encoding='latin-1')
pos_rew_pres={}
i=0
for line in t:
  dic1={}
  for word in line.split():
    if word not in dic1 and word.isalpha():
      dic1[word]=1
  dic1['label']=0
  pos_rew_pres[i]=dic1
  i=i+1
t.close()

In [0]:
t=open('rt-polarity.neg','r',encoding='latin-1')
neg_rew_pres={}
for line in t:
  dic1={}
  for word in line.split():
    if word not in dic1 and word.isalpha():
      dic1[word]=1
  dic1['label']=1
  neg_rew_pres[i]=dic1
  i=i+1
t.close()

In [0]:
f={**pos_rew_pres,**neg_rew_pres}
df1=pd.DataFrame.from_dict(f,orient='index')
df1.fillna(0,inplace=True)
a=df1.sum(axis=0)
a=a.to_dict()
list1=[]
for key,value in list(a.items()):
  if value<4:
    list1.append(key)
df1.drop(list1,axis=1,inplace=True)

In [0]:
##naive bayes
y1=df1.label
x1=df1.drop('label',axis=1)
x1_train,x1_test,y1_train,y1_test=train_test_split(x1,y1,test_size=0.3,random_state=1234,stratify=df1.label)
model2=GridSearchCV(MultinomialNB(),param_grid={},cv=10)
model2.fit(x1_train,y1_train)
y1_pred=model2.predict(x1_test)
accuracy_score(y1_test,y1_pred)

0.7661769302907159

In [0]:
##SVM
model3=GridSearchCV(SVC(),param_grid={},cv=10)
model3.fit(x1_train,y1_train)
y1_pred1=model3.predict(x1_test)
accuracy_score(y1_test,y1_pred1)

# creating the reviews without any proximity

In [0]:
g=list(df1.columns)
g.remove('label')

In [0]:
def makedataset(filename):
  t=open(r'/content/drive/My Drive/txt_sentoken/pos' + '//' + filename,'r')
  review=[]
  sent=[]
  for line in t:
    for word in line.split():
      if word=='.':
        review.append(sent)
        sent=[]
      elif word.isalpha():
        sent.append(word)
  t.close()
  return review

In [0]:
def min_cut_pos(list_sent):
  probs=[]
  line_sent=[]
  for i in range(len(list_sent)):
    dica={}
    for a in g:
      dica[a]=0
    for word in list_sent[i]:
      if word in dica and dica[word]==0:
        dica[word]=dica[word]+1
    df=pd.DataFrame.from_dict(dica,orient='index')
    df=df.transpose()
    pred=model2.predict_proba(df)
    probs.append(pred[0][0])
  top_idx = np.argsort(probs)[-5:]
  for i in range(len(list_sent)):
    if i in top_idx:
      line_sent.append(list_sent[i])
  return line_sent

In [0]:
def min_cut_neg(list_sent):
  dica={}
  for a in g:
    dica[a]=0
  for word in list_sent:
    if word in dica and dica[word]==0:
      dica[word]=dica[word]+1
  
  df=pd.DataFrame.from_dict(dica,orient='index')
  df=df.transpose()
  if model2.predict(df)==[[0]]:
    return False
  else:
    return True



In [0]:
r=open('pos1.txt','w')
count=0
for file in files:
  print(count)
  line_sent=makedataset(file)
  list_sent=min_cut_pos(line_sent)
  for i in range(len(list_sent)):
    for word in list_sent[i]:
      r.write(word)
      r.write('\t')
    r.write('\n')

  count=count+1
r.close()

In [0]:
r=open('neg1.txt','w')
for file in files1:
  t=open(r'/content/drive/My Drive/txt_sentoken/neg' + '//' + file,'r')
  line_sent=[]
  for line in t:
    for word in line.split():
      if word=='.':
        if min_cut(line_sent):
          for w in line_sent:
            r.write(w)
            r.write('\t')
        line_sent=[]
      elif word.isalpha():
        line_sent.append(word)
  r.write('\n')
  t.close()
r.close()

# Building the polarity classifier on new review set formed

In [0]:
t=open('pos1.txt','r')
pos_rew_pres1={}
i=0
for line in t:
  dic1={}
  for word in line.split():
    if word not in dic1 and word.isalpha():
      dic1[word]=1
  dic1['label']=0
  pos_rew_pres1[i]=dic1
  i=i+1
t.close()

In [0]:
t=open('neg1.txt','r')
neg_rew_pres1={}
for line in t:
  dic1={}
  for word in line.split():
    if word not in dic1 and word.isalpha():
      dic1[word]=1
  dic1['label']=1
  neg_rew_pres1[i]=dic1
  i=i+1
t.close()

In [0]:
f={**pos_rew_pres1,**neg_rew_pres1}
df2=pd.DataFrame.from_dict(f,orient='index')
df2.fillna(0,inplace=True)
a=df2.sum(axis=0)
a=a.to_dict()
list1=[]
for key,value in list(a.items()):
  if value<4:
    list1.append(key)
df2.drop(list1,axis=1,inplace=True)

In [0]:
y3=df2.label
x3=df2.drop('label',axis=1)
x3_train,x3_test,y3_train,y3_test=train_test_split(x3,y3,test_size=0.3,random_state=1234,stratify=df2.label)
model3=GridSearchCV(make_pipeline(MinMaxScaler(),MultinomialNB()),param_grid={'multinomialnb__alpha':[1]},cv=10)
model3.fit(x3_train,y3_train)
y3_pred=model3.predict(x3_test)
accuracy_score(y3_test,y3_pred)

0.7683333333333333

In [0]:
model5=GridSearchCV(SVC(),param_grid={},cv=3)
model5.fit(x3_train,y3_train)
y3_pred1=model5.predict(x3_test)
accuracy_score(y3_test,y3_pred1)

0.7766666666666666

# creating reviews with proximity function f(d)=1

In [0]:
def min_cut_pr(line_sent):
  dica={}
  list1=[]
  line1=[]
  line2=[]
  line3=[]
  list_sent=[]
  count=0
  for word in line_sent:
    if word=='.':
      count=count+1
    elif count==0:
      line1.append(word)
    elif count==1:
      line2.append(word)
    elif count==2:
      line3.append(word)

  for a in g:
    dica[a]=0
  for word in line_sent:
    if word in dica and dica[word]==0:
      dica[word]=1
    elif word=='.':
      df=pd.DataFrame.from_dict(dica,orient='index')
      df=df.transpose()
      list1.append((model2.predict_proba(df)).tolist())
  
  list1 = [item for sublist in list1 for item in sublist]
  list1 = [item for sublist in list1 for item in sublist]
  if len(list1)==6:
    if list1[0]>0.5 or list1[2]>0.5 or list1[4]>0.5:
      list_sent.append(line1)
      list_sent.append(line3)
      list_sent.append(line2)
  list_sent = [item for sublist in list_sent for item in sublist]
  
  return list_sent

In [0]:

###example to check the above function is working...
t=open('cv000_29590.txt','r')
line_sent=[]
count=0
for line in t:
  for word in line.split():
     if word=='.'and count==3:
        print(min_cut_pr(line_sent))
        line_sent=[]
        count=0
     elif word=='.':
       count=count+1 
       line_sent.append('.')   
     elif word.isalpha():
       line_sent.append(word)


In [0]:
r=open('pos2.txt','w')

for file in files:
  t=open(r'/content/drive/My Drive/txt_sentoken/pos' + '//' + file,'r')
  line_sent=[]
  for line in t:
    for word in line.split():
      if count==3:
        list_sent=min_cut_pr(line_sent)
        if list_sent!=[]:
          for w in list_sent:
            r.write(w)
            r.write('\t')
        line_sent=[]
        count=0
      elif word=='.':
        count=count+1 
        line_sent.append('.')   
      if word.isalpha():
        line_sent.append(word)
  r.write('\n')
  t.close()
  
r.close()

In [0]:
def min_cut_prne(line_sent):
  dica={}
  list1=[]
  line1=[]
  line2=[]
  line3=[]
  list_sent=[]
  count=0
  for word in line_sent:
    if word=='.':
      count=count+1
    elif count==0:
      line1.append(word)
    elif count==1:
      line2.append(word)
    elif count==2:
      line3.append(word)

  for a in g:
    dica[a]=0
  for word in line_sent:
    if word in dica and dica[word]==0:
      dica[word]=1
    elif word=='.':
      df=pd.DataFrame.from_dict(dica,orient='index')
      df=df.transpose()
      list1.append((model2.predict_proba(df)).tolist())
  
  list1 = [item for sublist in list1 for item in sublist]
  list1 = [item for sublist in list1 for item in sublist]
  if len(list1)==6:
    if list1[1]>0.5 or list1[3]>0.5 or list1[5]>0.5:
      list_sent.append(line1)
      list_sent.append(line3)
      list_sent.append(line2)
  list_sent = [item for sublist in list_sent for item in sublist]
  
  return list_sent

In [0]:
r=open('neg2.txt','w')
for file in files1:
  t=open(r'/content/drive/My Drive/txt_sentoken/neg' + '//' + file,'r')
  line_sent=[]
  for line in t:
    for word in line.split():
      if count==3:
        list_sent=min_cut_prne(line_sent)
        if list_sent!=[]:
          for w in list_sent:
            r.write(w)
            r.write('\t')
        line_sent=[]
        count=0
      elif word=='.':
        count=count+1 
        line_sent.append('.')   
      if word.isalpha():
        line_sent.append(word)
  r.write('\n')
  t.close()
r.close()

# Building the polarity classifier on new review set formed

In [0]:
t=open('pos2.txt','r')
pos_rew_pres2={}
i=0
for line in t:
  dic1={}
  for word in line.split():
    if word not in dic1 and word.isalpha():
      dic1[word]=1
  dic1['label']=0
  pos_rew_pres2[i]=dic1
  i=i+1
t.close()


In [0]:
t=open('neg2.txt','r')
neg_rew_pres2={}

for line in t:
  dic1={}
  for word in line.split():
    if word not in dic1 and word.isalpha():
      dic1[word]=1
  dic1['label']=1
  pos_rew_pres2[i]=dic1
  i=i+1
t.close()


In [0]:
f={**pos_rew_pres2,**neg_rew_pres2}
df3=pd.DataFrame.from_dict(f,orient='index')
df3.fillna(0,inplace=True)
a=df3.sum(axis=0)
a=a.to_dict()
list1=[]
for key,value in list(a.items()):
  if value<4:
    list1.append(key)
df3.drop(list1,axis=1,inplace=True)

In [0]:
y4=df3.label
x4=df3.drop('label',axis=1)
x4_train,x4_test,y4_train,y4_test=train_test_split(x4,y4,test_size=0.3,random_state=1234,stratify=df3.label)
model5=GridSearchCV(make_pipeline(MinMaxScaler(),MultinomialNB()),param_grid={'multinomialnb__alpha':[1]},cv=10)
model5.fit(x4_train,y4_train)
y4_pred=model5.predict(x4_test)
accuracy_score(y4_test,y4_pred)

0.875

In [0]:
model8=GridSearchCV(SVC(),param_grid={},cv=3)
model8.fit(x4_train,y4_train)
y4_pred1=model5.predict(x4_test)
accuracy_score(y4_test,y4_pred1)

0.875

# Creating the review documents with proximity function f(d)=1/d^2 or    f(d)=e^(1-d)

In [0]:
def min_cut_pra(line_sent):
  dica={}
  list1=[]
  line1=[]
  line2=[]
  line3=[]
  list_sent=[]
  count=0
  for word in line_sent:
    if word=='.':
      count=count+1
    elif count==0:
      line1.append(word)
    elif count==1:
      line2.append(word)
    elif count==2:
      line3.append(word)

  for a in g:
    dica[a]=0
  for word in line_sent:
    if word in dica and dica[word]==0:
      dica[word]=1
    elif word=='.':
      df=pd.DataFrame.from_dict(dica,orient='index')
      df=df.transpose()
      list1.append((model2.predict_proba(df)).tolist())
  
  list1 = [item for sublist in list1 for item in sublist]
  list1 = [item for sublist in list1 for item in sublist]
  if len(list1)==6:
    if list1[0]>0.5:
      list_sent.append(line1)
      list_sent.append(line2)
    elif list1[2]>0.5:
      list_sent.append(line1)
      list_sent.append(line2)
      list_sent.append(line3)
    elif list1[4]>0.5:
      list_sent.append(line2)
      list_sent.append(line3)
      
  list_sent = [item for sublist in list_sent for item in sublist]
  
  return list_sent

In [0]:
r=open('pos3.txt','w')
for file in files:
  t=open(r'/content/drive/My Drive/txt_sentoken/pos' + '//' + file,'r')
  line_sent=[]
  for line in t:
    for word in line.split():
      if count==3:
        list_sent=min_cut_pra(line_sent)
        if list_sent!=[]:
          for w in list_sent:
            r.write(w)
            r.write('\t')
        line_sent=[]
        count=0
      elif word=='.':
        count=count+1 
        line_sent.append('.')   
      if word.isalpha():
        line_sent.append(word)
  r.write('\n')
  t.close()
r.close()

In [0]:
def min_cut_prane(line_sent):
  dica={}
  list1=[]
  line1=[]
  line2=[]
  line3=[]
  list_sent=[]
  count=0
  for word in line_sent:
    if word=='.':
      count=count+1
    elif count==0:
      line1.append(word)
    elif count==1:
      line2.append(word)
    elif count==2:
      line3.append(word)

  for a in g:
    dica[a]=0
  for word in line_sent:
    if word in dica and dica[word]==0:
      dica[word]=1
    elif word=='.':
      df=pd.DataFrame.from_dict(dica,orient='index')
      df=df.transpose()
      list1.append((model2.predict_proba(df)).tolist())
  
  list1 = [item for sublist in list1 for item in sublist]
  list1 = [item for sublist in list1 for item in sublist]
  if len(list1)==6:
    if list1[0]>0.5:
      list_sent.append(line1)
      list_sent.append(line2)
    elif list1[2]>0.5:
      list_sent.append(line1)
      list_sent.append(line2)
      list_sent.append(line3)
    elif list1[4]>0.5:
      list_sent.append(line2)
      list_sent.append(line3)
      
  list_sent = [item for sublist in list_sent for item in sublist]
  
  return list_sent

In [0]:
r=open('neg3.txt','w')
for file in files1:
  t=open(r'/content/drive/My Drive/txt_sentoken/neg' + '//' + file,'r')
  line_sent=[]
  for line in t:
    for word in line.split():
      if count==3:
        list_sent=min_cut_prane(line_sent)
        if list_sent!=[]:
          for w in list_sent:
            r.write(w)
            r.write('\t')
        line_sent=[]
        count=0
      elif word=='.':
        count=count+1 
        line_sent.append('.')   
      if word.isalpha():
        line_sent.append(word)
  r.write('\n')
  t.close()
r.close()

# Building the polarity classifiers with new review documents

In [0]:
t=open('pos3.txt','r')
pos_rew_pres3={}
i=0
for line in t:
  dic1={}
  for word in line.split():
    if word not in dic1 and word.isalpha():
      dic1[word]=1
  dic1['label']=0
  pos_rew_pres3[i]=dic1
  i=i+1
t.close()


In [0]:
t=open('neg3.txt','r')
neg_rew_pres3={}
for line in t:
  dic1={}
  for word in line.split():
    if word not in dic1 and word.isalpha():
      dic1[word]=1
  dic1['label']=1
  neg_rew_pres3[i]=dic1
  i=i+1
t.close()

In [0]:
f={**pos_rew_pres3,**neg_rew_pres3}
df4=pd.DataFrame.from_dict(f,orient='index')
df4.fillna(0,inplace=True)
a=df4.sum(axis=0)
a=a.to_dict()
list1=[]
for key,value in list(a.items()):
  if value<4:
    list1.append(key)
df4.drop(list1,axis=1,inplace=True)

In [0]:
y5=df4.label
x5=df4.drop('label',axis=1)
x5_train,x5_test,y5_train,y5_test=train_test_split(x5,y5,test_size=0.3,random_state=1234,stratify=df4.label)
model6=GridSearchCV(make_pipeline(MinMaxScaler(),MultinomialNB()),param_grid={'multinomialnb__alpha':[1]},cv=10)
model6.fit(x5_train,y5_train)
y5_pred=model6.predict(x5_test)
accuracy_score(y5_test,y5_pred)

0.745

In [0]:
model5=GridSearchCV(SVC(),param_grid={},cv=3)
model5.fit(x5_train,y5_train)
y5_pred1=model5.predict(x5_test)
accuracy_score(y5_test,y5_pred1)

0.76